In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} -c conda-forge pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, size

In [ ]:
spark = SparkSession.builder.appName("Packt").getOrCreate()

In [ ]:
data = spark.read.csv('netflix_titles_nov_2019.csv', header='true')

In [ ]:
movies = data.filter((col('type') == 'TV Show') & ((col('rating') == 'TV-G') | (col('rating') == 'TV-Y')))

In [ ]:
transformed = movies.withColumn('count_lists', size(split(movies['listed_in'], ',')))

In [ ]:
selected = transformed.select('title', 'cast', 'rating', 'release_year', 'duration', 'count_lists', 'listed_in', 'description')

In [ ]:
selected.write.csv('transformed', header='true')

In [ ]:
!head transformed/part-00000-a9837c96-549d-4a8c-981a-cae147e36801-c000.csv 